# Use: Tests of `BenamouBrenier.py`

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from warnings import warn
from tqdm.notebook import tqdm,trange # to display loading bars
%load_ext autoreload
%autoreload 2

from BenamouBrenier import TransportProblem
from transport import gaussian_transport, gaussian_discreatization

rng = np.random.default_rng(4321)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
N=100
mesh= np.random.randn(2,N,N)
mu = np.random.randn(N,N)
nu = np.random.randn(N,N)
T = 10

prob1 = TransportProblem(mesh,mu,nu,T)
prob1.projection_step()
prob1.dual_step()

  0%|          | 0/100000 [00:00<?, ?it/s]

/Users/benoitmuller/Documents/GitHub/Computational-Optimal-Transport/BenamouBrenier.py:93: UserWarning: Max number of iterations reached in Newton's method.
  warn("Max number of iterations reached in Newton's method.")


KeyboardInterrupt: 

## Test Py-PDE

In [37]:

import pde
from pde import CartesianGrid, ScalarField, solve_poisson_equation, PDE
n=100
f = rng.normal(size=(n,n,n))
f = f - np.mean(f)
grid = pde.CartesianGrid([[0, 1], [0, 1],[0,1]], shape=(n, n,n), periodic=[True, True, True])
field = ScalarField(grid, f)
eq = PDE({"phi": "-gradient_squared(u) / 2 - laplace(u + laplace(u))"}) 
result = solve_poisson_equation(field, bc=["periodic", "periodic", "periodic"])

result.plot()

NotImplementedError: 3-dimensional Laplace matrix not implemented

## Todo
- check if code works for 1D ndarray with/without empty dimensions
- Use a package to compute poisson equation

## Remarks
- **Projection Step**:
    - Analytic solution complicated and numerically instable. reduce the problem to 1D (grid-wise), and use Newton method to solve the polynomial equation of orthogonality. [Algorithms for projecting points onto conics](https://www.sciencedirect.com/science/article/pii/S0377042713001398#s000005)
    - choice of the initial point ([desmos graphic](https://www.desmos.com/calculator/nj6gcjfbaq)) to get the right zero.
- **Dual Step**: easy
- **Poisson Step**:
    - first order condition by calculus of variations, how to do it, and leads to weak poisson equation.
    - border conditions
    - https://fenicsproject.org can solve poisson equation, see [demo_poisson](https://docs.fenicsproject.org/dolfinx/v0.6.0/python/demos/demo_poisson.html)
- What is the Domain $\Omega\subset\mathbb{R}^d$, should contain the support of $\mu$ and $\nu$?
- gaussian transport
    - restrict to an ellipsis support of $\mu$ and renormalise (polar coord and 1D normal distribution)
    - the transport map is still optimal and lead to an ellipsis support, and is still optimal.